In [1]:
import csv
import pandas as pd
import numpy as np
import re
import copy

In [2]:
n = 4
deny_list = ['0','1','2','3','4','5','6','7','8','9','，','？','。',
             '一','二','三','四','五','六','七','八','九','零','十',
            '的','小','请','.','?','有多少','帮我','我想','知道',
             '是多少','保留','是什么','-','(',')','（','）','：',
              '哪个','统计','且','和','来','请问','记得','有','它们']
pattern1 = r'\d{8}'

In [3]:
data_file_dir = '/mnt/workspace/intermediate/question_SQL_V6_exed.csv'
data_file = pd.read_csv(data_file_dir,delimiter = ",",header = 0)

In [4]:
data_file2_dir = '/mnt/workspace/intermediate/A01_question_classify.csv'
data_file2 = pd.read_csv(data_file2_dir,delimiter = ",",header = 0)

In [5]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

2024-07-04 19:30:42,981 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-07-04 19:30:42,982 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-04 19:30:43,010 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 3b674e250eaff3ed305bb955cabf2c95 and a total number of 980 components indexed
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# TDOO 原版用14B-Chat，太大加载不了，只能换Int4版本
model_dir = '/mnt/workspace/Tongyi-Finance-14B-Chat-Int4'
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True, 
                                             # bf16=True
                                            ).eval()

model.generation_config = GenerationConfig.from_pretrained(model_dir,
                                                           trust_remote_code=True,
                                                           temperature = 0.00001,
                                                           top_p = 1,
                                                           do_sample = False,
                                                           seed = 1234)
print('B03_model_loaded')

CUDA extension not installed.


B03_model_loaded


/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
deny_token_list = list()
for word in deny_list:
    temp_tokens = tokenizer(word)
    temp_tokens = temp_tokens['input_ids']
    deny_token_list = deny_token_list + temp_tokens

In [9]:
def get_prompt_v33(question,data,index_list):
    
    Examples = '以下是一些例子：'
    for index in index_list:
        Examples = Examples + "问题：" + example_question_list[index] + '\n'
        Examples = Examples + "资料：" + example_data_list[index] + '\n'
        Examples = Examples + "答案：" + example_FA_list[index] + '\n'
    impt2 = """
        你要进行句子生成工作，根据提供的资料来回答对应的问题。下面是一些例子。注意问题中对小数位数的要求。+ '\n'
    """
    
                
    impt2 = impt2 + Examples

    impt2 = impt2 +  "问题：" + question + '\n'
    impt2 = impt2 +  "资料：" + data + '\n'
    impt2 = impt2 +  "答案："
    return impt2

In [10]:
SQL_examples_file_dir = "/mnt/workspace/files/ICL_EXP.csv"
SQL_examples_file = pd.read_csv(SQL_examples_file_dir,delimiter = ",",header = 0)

In [11]:
example_employ_list = list()
for cyc in range(len(SQL_examples_file)):
    example_employ_list.append(0)

In [12]:
example_question_list = list()
example_data_list = list()
example_FA_list = list()
example_token_list = list()

for cyc in range(len(SQL_examples_file)):
    example_question_list.append(SQL_examples_file[cyc:cyc+1]['问题'][cyc])
    example_data_list.append(SQL_examples_file[cyc:cyc+1]['资料'][cyc])
    example_FA_list.append(SQL_examples_file[cyc:cyc+1]['FA'][cyc])
    temp_tokens = tokenizer(SQL_examples_file[cyc:cyc+1]['问题'][cyc])
    temp_tokens = temp_tokens['input_ids']
    temp_tokens2 = [x for x in temp_tokens if x not in deny_token_list]
    example_token_list.append(temp_tokens2)

In [13]:
g = open('/mnt/workspace/intermediate/FA_V5_SQL.csv', 'w', newline='', encoding = 'utf-8-sig') 
csvwriter = csv.writer(g)
csvwriter.writerow(['问题id','问题','FA','SQL结果'])

18

In [14]:
from tqdm import tqdm

for cyc in tqdm(range(1000)):
    if cyc % 50 == 0:
        print(cyc)
    temp_question = data_file[cyc:cyc+1]['问题'][cyc]
    class_ans = data_file2[cyc:cyc+1]['分类'][cyc]
    SQL_search_result = data_file[cyc:cyc+1]['执行结果'][cyc]
    temp_FA = temp_question
    if class_ans != 'SQL':
        temp_FA = 'N_A'
    elif SQL_search_result != 'N_A':
        if len(SQL_search_result) > 0:
            if len(SQL_search_result) > 250:
                SQL_search_result = SQL_search_result[0:250]
            temp_question = data_file[cyc:cyc+1]['问题'][cyc]
            date_list =  re.findall(pattern1,temp_question)
            temp_question2_for_search = temp_question
            for t_date in date_list:
                temp_question2_for_search.replace(t_date,' ')
            temp_tokens = tokenizer(temp_question2_for_search)
            temp_tokens = temp_tokens['input_ids']
            temp_tokens2 = [x for x in temp_tokens if x not in deny_token_list]
            temp_tokens = temp_tokens2
            #计算与已有问题的相似度
            similarity_list = list()
            for cyc2 in range(len(SQL_examples_file)):
                similarity_list.append(len(set(temp_tokens) &set(example_token_list[cyc2]))/ (len(set(temp_tokens))+len(set(example_token_list[cyc2])) ))

            #求与第X个问题相似的问题

            t = copy.deepcopy(similarity_list)
            # 求m个最大的数值及其索引
            max_number = []
            max_index = []
            for _ in range(n):
                number = max(t)
                index = t.index(number)
                t[index] = 0
                max_number.append(number)
                max_index.append(index)
            t = []
                
                
                
            prompt2 = get_prompt_v33(data_file['问题'][cyc],data_file['执行结果'][cyc],max_index)
            temp_FA, history = model.chat(tokenizer, prompt2, history=None)

            
    else:
        SQL_search_result = 'SQL未能成功执行！'
        
    
    csvwriter.writerow([str(data_file[cyc:(cyc+1)]['问题id'][cyc]),
                    str(data_file[cyc:(cyc+1)]['问题'][cyc]),temp_FA,SQL_search_result])

  0%|          | 0/1000 [00:00<?, ?it/s]/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


0


  5%|▍         | 49/1000 [11:08<2:06:26,  7.98s/it]

50


 10%|█         | 100/1000 [21:28<2:14:17,  8.95s/it]

...

 90%|█████████ | 900/1000 [3:14:28<35:36, 21.36s/it]

900


 95%|█████████▌| 950/1000 [3:23:51<06:35,  7.91s/it]

950


100%|██████████| 1000/1000 [3:34:21<00:00, 12.86s/it]


In [15]:
g.close()